In [1]:
import json
import pandas as pd
from simple_salesforce import Salesforce, format_soql, format_external_id
from json import loads, dumps
import simpler_sf
simpler_sf.simple_salesforce()
from sqlalchemy import create_engine

In [ ]:
with open("../login.json", "r") as j:
    misDatos = json.load(j)
sf = Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)

In [48]:
#usuario = 'siempreread'
#contrasena = 'S13mpr3_Ch1l3'
#url_servidor = '209.151.148.250'
puerto = '3306'
esquema = 'SiempreChileNAB'
plugin_autenticacion = 'mysql_native_password'

usuario = 'root'
contrasena = 'Ecd1h1cdp!'
url_servidor = '10.0.10.24'

# Conexión a la base de datos

c_conexion = 'mysql+mysqlconnector://{0}:{1}@{2}:{3}/{4}?auth_plugin={5}'
c_conexion = c_conexion.format(usuario, contrasena, url_servidor, puerto,
                               esquema, plugin_autenticacion)
motor_mysql_mariadb = create_engine(c_conexion)



In [44]:
print (url_servidor)

10.0.10.24


In [49]:
listado_email = 'SELECT * FROM SiempreChileNAB.registro_nab LIMIT 10'
emails = pd.read_sql_query(listado_email, motor_mysql_mariadb)

In [50]:
emails.head()

,id,ciudad,codigo_carga,codigo_postal,comuna,convenio,convenio_plan,direccion,fecha_bloqueo,fecha_modificacion,...,p_operacion2,p_paterno,p_poliza,p_relacion,p_rut_beneficiario,p_rut_titular,p_sexo,procesado,rut_empresa,version
0,1,,0,,,77,SIN PLAN,,0,2022-03-11,...,A,CAMAYA,340014823,1,88002059,88002059,2,0,761223380,1
1,2,,1,,,77,SIN PLAN,,0,2022-03-11,...,A,EIZAGUIRRE,340014823,2,90023349,88002059,1,0,761223380,1
2,3,,2,,,77,SIN PLAN,,0,2022-03-11,...,A,EIZAGUIRRE,340014823,3,215736792,88002059,1,0,761223380,1
3,4,,1,,,77,SIN PLAN,,0,2022-03-11,...,A,TORREJJON,340019096,4,39236184,80048602,1,0,969546906,1
4,5,,0,,,77,SIN PLAN,,0,2022-03-11,...,A,HUERTASA,340017809,1,188694888,188694888,2,0,967838802,1


In [ ]:
query = "select Id, Account.Name, RutContact__c from AccountContactRelation "

df1 = sf.smart_query(query)